In [ ]:
# Evaluación Unidad III - Data Mining
# Autor: [Tu Nombre]
# Fecha: [Fecha Actual]
# Docente: Ing. Genoveva Suing Albito

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report

# Estilo de gráficos
sns.set(style="whitegrid")

# 🔹 Clase SimuladorDatos
class SimuladorDatos:
    def __init__(self, n=200, seed=42):
        self.n = n
        self.seed = seed
        np.random.seed(self.seed)

    def generar(self):
        edad = np.random.randint(18, 31, size=self.n)
        horas_estudio = np.random.uniform(0, 30, size=self.n).round(2)
        asistencia = np.random.uniform(50, 100, size=self.n).round(2)
        promedio = np.random.uniform(5, 10, size=self.n).round(2)
        uso_online = np.random.uniform(0, 15, size=self.n).round(2)
        abandono = np.random.choice([0, 1], size=self.n, p=[0.7, 0.3])

        df = pd.DataFrame({
            'Edad': edad,
            'Horas_estudio': horas_estudio,
            'Asistencia': asistencia,
            'Promedio': promedio,
            'Uso_online': uso_online,
            'Abandono': abandono
        })

        return df

# 🔹 Clase ModeloAbandono
class ModeloAbandono:
    def __init__(self, max_depth=4, random_state=42):
        self.max_depth = max_depth
        self.random_state = random_state
        self.modelo = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)

    def entrenar(self, data):
        self.data = data
        X = data.drop('Abandono', axis=1)
        y = data['Abandono']
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=0.3, random_state=self.random_state
        )
        self.modelo.fit(self.X_train, self.y_train)

    def evaluar(self):
        y_pred = self.modelo.predict(self.X_test)
        acc = accuracy_score(self.y_test, y_pred)
        print(f"\n🔍 Accuracy del modelo: {acc:.2f}")
        print("\n📋 Reporte de clasificación:\n")
        print(classification_report(self.y_test, y_pred))

        # Visualización del árbol
        plt.figure(figsize=(12, 6))
        plot_tree(self.modelo, feature_names=self.X_train.columns, class_names=["Continúa", "Abandona"], filled=True)
        plt.title("🌳 Árbol de Decisión Entrenado")
        plt.show()

    def predecir_estudiante(self, estudiante_df):
        pred = self.modelo.predict(estudiante_df)[0]
        return "Abandonará" if pred == 1 else "Seguirá estudiando"

# 🔹 Clase TestBasicoModeloAbandono
class TestBasicoModeloAbandono:
    def ejecutar(self):
        # 1. Generar los datos
        print("✅ Generando datos simulados...")
        simulador = SimuladorDatos()
        datos = simulador.generar()
        print(datos.head())

        # Visualización inicial
        self.visualizar_datos(datos)

        # 2. Entrenar el modelo
        print("\n✅ Entrenando el modelo de abandono...")
        modelo = ModeloAbandono()
        modelo.entrenar(datos)

        # 3. Evaluar el modelo
        print("\n✅ Evaluando el modelo...")
        modelo.evaluar()

        # 4. Predecir un nuevo estudiante
        print("\n✅ Predicción de un nuevo estudiante:")
        nuevo_estudiante = pd.DataFrame({
            'Edad': [21],
            'Horas_estudio': [10.5],
            'Asistencia': [85.0],
            'Promedio': [8.0],
            'Uso_online': [6.5]
        })
        resultado = modelo.predecir_estudiante(nuevo_estudiante)
        print(f"\n🎓 Resultado de la predicción: {resultado}")

    def visualizar_datos(self, df):
        # Conteo de abandono
        plt.figure(figsize=(6, 4))
        sns.countplot(x='Abandono', data=df, palette='Set2')
        plt.title("📊 Distribución de estudiantes que abandonan vs continúan")
        plt.xticks([0, 1], ['Continúa', 'Abandona'])
        plt.ylabel("Cantidad")
        plt.xlabel("Estado")
        plt.show()

        # Gráfico de dispersión Horas_estudio vs Promedio
        plt.figure(figsize=(7, 5))
        sns.scatterplot(data=df, x='Horas_estudio', y='Promedio', hue='Abandono', palette='Set1')
        plt.title("🎯 Relación entre Horas de Estudio y Promedio")
        plt.xlabel("Horas de estudio por semana")
        plt.ylabel("Promedio académico")
        plt.legend(title="Abandono", labels=["Continúa", "Abandona"])
        plt.show()

        # Mapa de calor de correlación
        plt.figure(figsize=(8, 6))
        sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
        plt.title("📈 Matriz de correlación de variables")
        plt.show()

# ▶️ Ejecutar todo el flujo
if __name__ == "__main__":
    test = TestBasicoModeloAbandono()
    test.ejecutar()
